### Tasks 

Here we have an yet unpublished dataset on protein concentrations in the supernatant of a CHO-cell bioreactor for protein concentrations. The concentrations have been determined by so-called iBAQ spectral counting.

* Download the data set.
    * Read in the data to a pandas data frame. Tip: use a command like df = pd.read_excel("CHOPER_IBAQ_sep.xlsx",header=0,index_col=0)
    * remove lines with NaN or 0-values.
    * Test which proteins that are differentially expressed between day 10and 30 at FDR 1% using a regular t-test e.g. scipy.stats.ttest_ind
    * (*) Test differential expression over time for each protein using a pandas model “Expression ~ C(Day)”.
    * (**) Compare your results to a linear regression model, e.g. “Expression ~ Day”

In [39]:
import os 
import numpy as np
import numpy.random as npr
import pandas as pd

os.chdir("../data/")
df = pd.read_excel("CHOPER_IBAQ_sep.xlsx",header=0,index_col=0)
df.loc["Day",:] = df.loc["Day",:].apply(int)
## Find the nan in dataframe
print (df.isnull().sum().sum())


2754


In [35]:
df.replace([np.inf, -np.inf,0.0], np.nan)
df.dropna(inplace=True,axis=0)
df.shape

(2489, 51)

In [31]:
df.head()

,iBAQ 1 (Day 3 r1),iBAQ 2 (Day 3 r2),iBAQ 3 (Day 3 r3),iBAQ 4 (Day 5 r1),iBAQ 5 (Day 5 r2),iBAQ 6 (Day 5 r3),iBAQ 7 (Day 10 r1),iBAQ 8 (Day 10 r2),iBAQ 9 (Day 10 r3),iBAQ 10 (Day 12 r1),...,iBAQ 42 (Day 39 r3),iBAQ 43 (Day 40 r1),iBAQ 44 (Day 40 r2),iBAQ 45 (Day 40 r3),iBAQ 46 (Day 41 r1),iBAQ 47 (Day 41 r2),iBAQ 48 (Day 41 r3),iBAQ 49 (Day 45 r1),iBAQ 50 (Day 45 r2),iBAQ 51 (Day 45 r3)
Protein IDs,,,,,,,,,,,,,,,,,,,,,
Day,3.000000e+00,3.000000e+00,3.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,10.0,10.0,10.0,12.0,...,3.900000e+01,4.000000e+01,4.000000e+01,4.000000e+01,4.100000e+01,4.100000e+01,4.100000e+01,4.500000e+01,4.500000e+01,4.500000e+01
CON__Q3MHN5;CON__ENSEMBL:ENSBTAP00000018229,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.663600e+05,0.000000e+00,0.0,0.0,0.0,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
CON__ENSEMBL:ENSBTAP00000038253,0.000000e+00,0.000000e+00,1.534900e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
CON__P00761,2.388700e+09,3.080300e+09,3.022600e+09,2.878500e+09,2.832100e+09,2.674700e+09,837270000.0,478160000.0,656560000.0,743860000.0,...,2.065000e+09,1.440400e+09,2.167600e+09,2.618300e+09,2.933300e+09,1.424800e+09,2.518100e+09,3.003700e+09,1.504400e+09,1.563300e+09
CON__P02533;tr|G3I8F9|G3I8F9_CRIGR;CON__Q6IFX2;tr|G3I8G1|G3I8G1_CRIGR;CON__A2A4G1;CON__P08727;tr|G3I8F7|G3I8F7_CRIGR;CON__P19001;CON__Q61782;tr|G3INZ7|G3INZ7_CRIGR;CON__P35900;CON__Q9D312;CON__P05784;CON__Q99456;tr|G3IPB6|G3IPB6_CRIGR;tr|G3IPJ0|G3IPJ0_CRIGR;tr|G3HX38|G3HX38_CRIGR;tr|G3IPC7|G3IPC7_CRIGR;CON__Q8N1A0;CON__Q14525;CON__Q9UE12;CON__Q15323;CON__A2A5Y0;CON__Q14532;CON__A2AB72;CON__Q497I4;CON__O76015;CON__O76013;CON__Q7Z3Y9;CON__O76014;CON__REFSEQ:XP_986630;tr|G3I8F3|G3I8F3_CRIGR;tr|G3I8F1|G3I8F1_CRIGR,3.406000e+06,4.858500e+06,5.641600e+06,6.155600e+06,7.301600e+06,5.504800e+06,1560600.0,995070.0,1838200.0,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.326300e+05,5.457200e+05,0.000000e+00,0.000000e+00,0.000000e+00


In [36]:
2754-2489

265

In [41]:
def bootstrap(invec):
    idx = npr.randint(0, len(invec), len(invec))
    return [invec[i] for i in idx]

def estimatePi0(p, numBoot=100, numLambda=100, maxLambda=0.95):
    p.sort()
    n=len(p)
    lambdas=np.linspace(maxLambda/numLambda,maxLambda,numLambda)
    Wls=np.array([n-np.argmax(p>=l) for l in lambdas])
    pi0s=np.array([Wls[i] / (n * (1 - lambdas[i])) for i in range(numLambda)])
    minPi0=np.min(pi0s)
    mse = np.zeros(numLambda)
    for boot in range(numBoot):
        pBoot = bootstrap(p)
        pBoot.sort()
        WlsBoot =np.array([n-np.argmax(pBoot>=l) for l in lambdas])
        pi0sBoot =np.array([WlsBoot[i] / (n *(1 - lambdas[i])) for i in range(numLambda)])
        mse = mse + np.square(pi0sBoot-minPi0)
    minIx = np.argmin(mse)
    return pi0s[minIx]

def qvalues(pvalues):
    m=len(pvalues)
    pvalues.sort()
    pi0 = estimatePi0([p for p,coord in pvalues])
    num_p, p_sum, qs = 0.0, 0.0, []
    for p,coord in pvalues:
        num_p += 1.0
        p_sum += p
        q = pi0*p*m/num_p
        qs.append((q,p,coord))
    qs.reverse()
    old_q=1.0
    for ix in range(len(qs)):
        q = min(old_q,qs[ix][0])
        old_q = q
        qs[ix] = (q,qs[ix][1],qs[ix][2])
    qs.reverse()
    return qs

In [81]:
df.loc[:,("iBAQ 7 (Day 10 r1)","iBAQ 8 (Day 10 r2)","iBAQ 34 (Day 30 r1)","iBAQ 35 (Day 30 r2) ","iBAQ 36 (Day 30 r3)")].values

array([[  1.00000000e+01,   1.00000000e+01,   1.00000000e+01,
          3.00000000e+01,   3.00000000e+01,   3.00000000e+01],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       ..., 
       [  5.01570000e+06,   1.92180000e+06,   3.06180000e+06,
          1.08400000e+07,   3.09720000e+06,   3.47370000e+06],
       [  4.39830000e+06,   2.27270000e+06,   3.94650000e+06,
          3.79780000e+06,   3.14230000e+06,   2.46540000e+06],
       [  9.95660000e+06,   3.82950000e+06,   2.79800000e+06,
          1.01970000e+07,   2.86850000e+06,   2.23220000e+06]])

In [87]:
df.loc[:,("iBAQ 7 (Day 10 r1)", "iBAQ 8 (Day 10 r2)")].mean()

iBAQ 7 (Day 10 r1)    1.107961e+07
iBAQ 8 (Day 10 r2)    6.627495e+06
dtype: float64